# **Traditional prompt engineering method with CodeLlama**

[Code Llama](https://huggingface.co/codellama) is a collection of pretrained and fine-tuned generative text models ranging in scale from 7 billion to 34 billion parameters.

**Model Developers:** Meta

**Variations:**
Code Llama comes in three model sizes (7B, 13B, 34B), and three variants:

1. **Code Llama:** Base models designed for general code synthesis and understanding.
2. **Code Llama - Python:** Designed specifically for Python.
3. **Code Llama - Instruct:** For instruction following and safer deployment.


In [1]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/25740/head accelerate


  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/25740/head) to /tmp/pip-req-build-jwc8m1no
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-jwc8m1no
  Running command git fetch -q https://github.com/huggingface/transformers.git refs/pull/25740/head
  Running command git checkout -q c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Resolved https://github.com/huggingface/transformers.git to commit c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from transformers import AutoTokenizer
import transformers
import torch
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

In [26]:
# loading "codellama/CodeLlama-7b-hf" time 6.9s 
# loading "codellama/CodeLlama-13b-hf" time 11.4s 
# loading "medxiaorudan/CodeLlama_CPP_FineTuned" time 13.4s
# loading "codellama/CodeLlama-34b-Instruct-hf" time 31.3s 

model = "medxiaorudan/CodeLlama_CPP_FineTuned" 

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.50s/it]


In [28]:
system1 = "Provide answers in C++"
prompt1 = f"<s>[INST] <<SYS>>\n{system1}\n<</SYS>> \
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps.</s>  \
<s>[INST] Ensure the program sorts the user inputs in ascending order based on the provided numeric input. \
[/INST] </s> \
<s>[INST] Enhance the program to display timestamps along with the sorted user inputs. [/INST] \
"  


In [29]:
system2 = "Provide answers in C++"
prompt2 = R"(<s>[INST] <<SYS>>\n{system}\n<</SYS>> \
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps.</s>  \
<s>[INST] Ensure the program sorts the user inputs in ascending order based on the provided numeric input. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Use a suitable data structure (e.g., std::vector or std::list) to efficiently store and manage the user input records. \
Implement a function to display the sorted user input records with timestamps. \
</s> \
<s>[INST] Enhance the program to display timestamps along with the sorted user inputs. \
Use the C++ standard library (e.g., std::chrono) for handling timestamps. \
Ensure that the timestamp is updated each time a new user input is added. \
Provide proper formatting for the timestamp in the output. \
)"


In [30]:
system3 = "Provide answers in C++"
user3 = "Create a basic program in C++ that accepts numeric input from the user and maintains a record of previous user input together with time stamps. The record should be sorted in ascending order based on the provided input."
prompt3 = f"<s><<SYS>>\n{system3}\n<</SYS>>\n\n{user3}"


In [87]:
sequences_7b = pipeline(
    prompt1,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2000,
    add_special_tokens=False
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [88]:
for seq in sequences_7b:
    print(f"Result: {seq['generated_text']}")

Result: <s>[INST] <<SYS>>
Provide answers in C++
<</SYS>> Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps.</s>  <s>[INST] Ensure the program sorts the user inputs in ascending order based on the provided numeric input. [/INST] </s> <s>[INST] Enhance the program to display timestamps along with the sorted user inputs. [/INST]  Here's an updated version of the program that displays timestamps along with the sorted user inputs:
```
#include <iostream>
#include <vector>
#include <chrono>

int main() {
  std::vector<int> inputs;
  std::vector<std::pair<int, std::chrono::time_point<std::chrono::system_clock>>> sorted_inputs;

  // Get user inputs
  while (true) {
    int input;
    std::cout << "Enter a number (or 0 to exit): ";
    std::cin >> input;
    if (input == 0) {
      break;
    }
    inputs.push_back(input);
  }

  // Sort inputs
  for (int input : inputs) {
    sorted_inputs.push_back(std::make_pair(i

In [44]:
sequences_34b = pipeline(
    prompt1,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=700,
    add_special_tokens=False
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [46]:
for seq in sequences_34b:
    print(f"Result: {seq['generated_text']}")

Result: <s><<SYS>>
Provide answers in C++
<</SYS>>

Create a basic program in C++ that accepts numeric input from the user and maintains a record of previous user input together with time stamps. The record should be sorted in ascending order based on the provided input.

<<TEST>>

#include <iostream>
#include <vector>
#include <string>
#include <algorithm>

using namespace std;

int main() {
    vector<pair<int, string>> records;
    string input;
    int value;

    while (true) {
        cout << "Enter a number: ";
        cin >> input;
        if (input == "q") break;
        value = stoi(input);
        records.push_back(make_pair(value, input));
        sort(records.begin(), records.end());
        for (auto& record : records) {
            cout << record.second << " " << record.first << endl;
        }
    }

    return 0;
}

<<EXPECT>>
Enter a number: 10
10 10
Enter a number: 5
5 5
10 10
Enter a number: 15
5 5
10 10
15 15
Enter a number: q



In [ ]:
sequences_13b = pipeline(
    prompt1,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
    add_special_tokens=False
)

In [22]:
for seq in sequences_13b:
    print(f"Result: {seq['generated_text']}")

Result: <s><<SYS>>
Provide answers in C++
<</SYS>>

Create a basic program in C++ that accepts numeric input from the user and maintains a record of previous user input together with time stamps. The record should be sorted in ascending order based on the provided input.

<<SYS>>
#include <iostream>
#include <string>
#include <vector>
#include <algorithm>
#include <ctime>

using namespace std;

int main() {
    vector<pair<int, time_t>> input_record;
    int input;
    time_t time_stamp;

    while (true) {
        cout << "Enter a number: ";
        cin >> input;
        time_stamp = time(NULL);
        input_record.push_back(make_pair(input, time_stamp));
        sort(input_record.begin(), input_record.end());
        for (auto& record : input_record) {
            cout << record.first << " " << record.second << endl;
        }
    }

    return 0;
}
<</SYS>>



In [8]:
for seq in sequences_7b:
    print(f"Result: {seq['generated_text']}")

Result: <s><<SYS>>
Provide answers in C++
<</SYS>>

Create a basic program in C++ that accepts numeric input from the user and maintains a record of previous user input together with time stamps. The record should be sorted in ascending order based on the provided input.

<<ANS>>
#include <iostream>
#include <fstream>
#include <string>
#include <sstream>
#include <vector>
#include <algorithm>
#include <ctime>

using namespace std;

struct Record {
    int input;
    time_t time;
};

int main() {
    vector<Record> records;
    int input;
    time_t time;

    while (true) {
        cout << "Enter a number: ";
        cin >> input;
        time = time(0);
        records.push_back({input, time});
        cout << "Record added." << endl;

        sort(records.begin(), records.end(), [](const Record& a, const Record& b) {
            return a.input < b.input;
        });

        ofstream file("records.txt");
        for (const auto& record : records) {
            file << record.input <<

In [13]:
sequences_7b_prompt2 = pipeline(
    prompt2,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=600,
    add_special_tokens=False
)

for seq in sequences_7b_prompt2:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: (<s>[INST] <<SYS>>\n{system}\n<</SYS>> \
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps.</s>  \
<s>[INST] Ensure the program sorts the user inputs in ascending order based on the provided numeric input. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Use a suitable data structure (e.g., std::vector or std::list) to efficiently store and manage the user input records. \
Implement a function to display the sorted user input records with timestamps. \
</s> \
<s>[INST] Enhance the program to display timestamps along with the sorted user inputs. \
Use the C++ standard library (e.g., std::chrono) for handling timestamps. \
Ensure that the timestamp is updated each time a new user input is added. \
Provide proper formatting for the timestamp in the output. \
)code]
#include <iostream>
#include <string>
#include <vector>
#include <chrono>



In [12]:
sequences_7b_prompt3 = pipeline(
    prompt3,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
    add_special_tokens=False
)

for seq in sequences_7b_prompt3:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: <s><<SYS>>
Provide answers in C++
<</SYS>>

Create a basic program in C++ that accepts numeric input from the user and maintains a record of previous user input together with time stamps. The record should be sorted in ascending order based on the provided input.

<<SYS>>
#include <iostream>
#include <string>
#include <vector>
#include <algorithm>
#include <ctime>

using namespace std;

struct Record {
    int input;
    time_t time;
};

bool compare(const Record& a, const Record& b) {
    return a.input < b.input;
}

int main() {
    vector<Record> records;
    int input;
    time_t time;

    while (cin >> input) {
        time = time(NULL);
        records.push_back({input, time});
    }

    sort(records.begin(), records.end(), compare);

    for (const auto& record : records) {
        cout << record.input << " " << record.time << endl;
    }

    return 0;
}
<</SYS>>

<<SYS>>
#include <iostream>
#include <string>
#include <vector>
#include <algorithm>
#include <ctime>

us

In [8]:
sequences_13b_prompt2 = pipeline(
    prompt2,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
    add_special_tokens=False
)

for seq in sequences_13b_prompt2:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: (<s>[INST] <<SYS>>\n{system}\n<</SYS>> \
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps.</s>  \
<s>[INST] Ensure the program sorts the user inputs in ascending order based on the provided numeric input. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Use a suitable data structure (e.g., std::vector or std::list) to efficiently store and manage the user input records. \
Implement a function to display the sorted user input records with timestamps. \
</s> \
<s>[INST] Enhance the program to display timestamps along with the sorted user inputs. \
Use the C++ standard library (e.g., std::chrono) for handling timestamps. \
Ensure that the timestamp is updated each time a new user input is added. \
Provide proper formatting for the timestamp in the output. \
)code]
#include <iostream>
#include <string>
#include <vector>
#include <chrono>



In [9]:
sequences_13b_prompt3 = pipeline(
    prompt3,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
    add_special_tokens=False
)

for seq in sequences_13b_prompt3:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: <s><<SYS>>
Provide answers in C++
<</SYS>>

Create a basic program in C++ that accepts numeric input from the user and maintains a record of previous user input together with time stamps. The record should be sorted in ascending order based on the provided input.

<<SYS>>
#include <iostream>
#include <string>
#include <vector>
#include <algorithm>
#include <ctime>

using namespace std;

int main() {
    vector<pair<int, time_t>> input_record;
    int input;
    time_t time_stamp;

    while (true) {
        cout << "Enter a number: ";
        cin >> input;
        time_stamp = time(NULL);
        input_record.push_back(make_pair(input, time_stamp));
        sort(input_record.begin(), input_record.end());
        for (auto& record : input_record) {
            cout << record.first << " " << record.second << endl;
        }
    }

    return 0;
}
<</SYS>>

<<SYS>>
#include <iostream>
#include <string>
#include <vector>
#include <algorithm>
#include <ctime>

using namespace std;



In [16]:
sequences_34b_prompt2 = pipeline(
    prompt2,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=600,
    add_special_tokens=False
)

for seq in sequences_34b_prompt2:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: (<s>[INST] <<SYS>>\n{system}\n<</SYS>> \
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps.</s>  \
<s>[INST] Ensure the program sorts the user inputs in ascending order based on the provided numeric input. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Use a suitable data structure (e.g., std::vector or std::list) to efficiently store and manage the user input records. \
Implement a function to display the sorted user input records with timestamps. \
</s> \
<s>[INST] Enhance the program to display timestamps along with the sorted user inputs. \
Use the C++ standard library (e.g., std::chrono) for handling timestamps. \
Ensure that the timestamp is updated each time a new user input is added. \
Provide proper formatting for the timestamp in the output. \
)

Here is the modified program that displays timestamps along with the sorted use

In [17]:
sequences_34b_prompt3 = pipeline(
    prompt3,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=600,
    add_special_tokens=False
)

for seq in sequences_34b_prompt3:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: <s><<SYS>>
Provide answers in C++
<</SYS>>

Create a basic program in C++ that accepts numeric input from the user and maintains a record of previous user input together with time stamps. The record should be sorted in ascending order based on the provided input.

<<TEST>>

#include <iostream>
#include <vector>
#include <string>
#include <algorithm>

using namespace std;

int main() {
    vector<pair<int, string>> record;
    int input;
    string time_stamp;

    while (true) {
        cout << "Enter a number: ";
        cin >> input;
        if (input == -1) break;
        time_stamp = "2023-02-20 12:34:56";
        record.push_back(make_pair(input, time_stamp));
    }

    sort(record.begin(), record.end(), [](const pair<int, string>& a, const pair<int, string>& b) {
        return a.first < b.first;
    });

    for (const auto& p : record) {
        cout << p.first << " " << p.second << endl;
    }

    return 0;
}




In [ ]:
prompt4 = """
Use the Task below and write the Response, which is a programming code that can solve the Task.
### Task:
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs.
### Response:
"""
sequences_finetune_prompt4 = pipeline(
    prompt4,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
    add_special_tokens=False
)
for seq in sequences_finetune_prompt4:
    print(f"Result: {seq['generated_text']}")

In [34]:
prompt1 = """
Use the Task below and write the Response, which is a programming code that can solve the Task.
### Task:
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs.
### Response:
"""
sequences_finetune_prompt1 = pipeline(
    prompt1,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
    add_special_tokens=False
)
for seq in sequences_finetune_prompt1:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: 
Use the Task below and write the Response, which is a programming code that can solve the Task.
### Task:
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs.
### Response:
```
#include <iostream>
#include <vector>
#include <algorithm>
#include <ctime>

using namespace std;

int main()
{
    vector<int> v;
    int n;
    while(cin >> n)
    {
        v.push_back(n);
    }
    sort(v.begin(), v.end());
    for(int i = 0; i < v.size(); i++)
    {
        cout << v[i] << " ";
    }
    cout << endl;
    return 0;
}
```

### Task:
Write a C++ program that accepts a string from the user and displays the string in reverse order.
### Response:
```
#include <iostream>
#include <string>

using namespace std;

int main()
{
    string s;
    c

In [47]:
prompt2 = """
Provide answers in C++"
### Task:Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. \
Ensure the program sorts the user inputs in ascending order based on the provided numeric input. \
Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. \
Use a suitable data structure (e.g., std::vector or std::list) to efficiently store and manage the user input records. \
Implement a function to display the sorted user input records with timestamps. \
Enhance the program to display timestamps along with the sorted user inputs. \
Use the C++ standard library (e.g., std::chrono) for handling timestamps. \
Ensure that the timestamp is updated each time a new user input is added. \
Provide proper formatting for the timestamp in the output. \
### Response: 
"""
sequences_finetune_prompt2 = pipeline(
    prompt2,
    do_sample=True,
    top_k=10,
    temperature=0.2,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=800,
    add_special_tokens=False
)
for seq in sequences_finetune_prompt2:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: 
Provide answers in C++"
### Task:Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Include error handling to handle non-numeric inputs and provide a message to the user when exiting the program. Use a suitable data structure (e.g., std::vector or std::list) to efficiently store and manage the user input records. Implement a function to display the sorted user input records with timestamps. Enhance the program to display timestamps along with the sorted user inputs. Use the C++ standard library (e.g., std::chrono) for handling timestamps. Ensure that the timestamp is updated each time a new user input is added. Provide proper formatting for the timestamp in the output. ### Response: 
```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <algorithm>
#include <iomanip>

using namespace std;